# Random forest

A supervised machine learning model that uses multiple decision trees in aggregate to predict the direction of the stock movements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas_datareader import DataReader as pdr
from datetime import datetime
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [2]:
symbols = ['BILI', 'PDD', 'JPM', 'WMT', 'AAPL']

# define the timeframe 
end = datetime.now()
start = end - relativedelta(years=2)

# retive data from yahoo finance
stocks = pd.DataFrame()

stocks = pdr(symbols, data_source='yahoo', start=start.date(), end=end.date())

In [3]:
# calculate the change in price for each stock
for symbol in stocks.columns.get_level_values(1)[:5]:
    stocks['price_change', symbol] = stocks['Close'][symbol].diff()

stocks.dropna(inplace=True)
stocks

Attributes   Adj Close                                                  \
Symbols           BILI         PDD         JPM         WMT        AAPL   
Date                                                                     
2019-05-22   14.690000   20.459999  103.975410   98.809914   44.853859   
2019-05-23   14.320000   20.530001  101.930054   98.452286   44.088215   
2019-05-24   14.050000   20.280001  102.933960   99.235184   43.918888   
2019-05-28   14.010000   20.230000  101.817474   98.993553   43.737293   
2019-05-29   13.780000   19.980000  101.536003   98.703583   43.528706   
...                ...         ...         ...         ...         ...   
2021-05-17   98.239998  118.019997  164.669998  138.889999  126.269997   
2021-05-18  101.059998  122.239998  162.350006  141.910004  124.849998   
2021-05-19  102.459999  127.169998  161.110001  141.919998  124.690002   
2021-05-20  102.800003  131.000000  160.830002  142.419998  127.309998   
2021-05-21  102.419998  129.679993  162.250000  141.828293  125.569099   

Attributes       Close                                                  ...  \
Symbols           BILI         PDD         JPM         WMT        AAPL  ...   
Date                                                                    ...   
2019-05-22   14.690000   20.459999  110.820000  102.230003   45.695000  ...   
2019-05-23   14.320000   20.530001  108.639999  101.860001   44.915001  ...   
2019-05-24   14.050000   20.280001  109.709999  102.669998   44.742500  ...   
2019-05-28   14.010000   20.230000  108.519997  102.419998   44.557499  ...   
2019-05-29   13.780000   19.980000  108.220001  102.120003   44.345001  ...   
...                ...         ...         ...         ...         ...  ...   
2021-05-17   98.239998  118.019997  164.669998  138.889999  126.269997  ...   
2021-05-18  101.059998  122.239998  162.350006  141.910004  124.849998  ...   
2021-05-19  102.459999  127.169998  161.110001  141.919998  124.690002  ...   
2021-05-20  102.800003  131.000000  160.830002  142.419998  127.309998  ...   
2021-05-21  102.419998  129.679993  162.250000  141.828293  125.569099  ...   

Attributes     Volume                                                   \
Symbols          BILI         PDD         JPM         WMT         AAPL   
Date                                                                     
2019-05-22  2483000.0  10456400.0   7918900.0   7340900.0  118994400.0   
2019-05-23  4847400.0   8147200.0  14558000.0   6454600.0  146118800.0   
2019-05-24  2452900.0   3124200.0   8537300.0   4483200.0   94858800.0   
2019-05-28  3617400.0   7613600.0  11665500.0  10271700.0  111792800.0   
2019-05-29  3338000.0   5740300.0   9966400.0   6520200.0  113924800.0   
...               ...         ...         ...         ...          ...   
2021-05-17  4564300.0   4150800.0   9886800.0   6558600.0   74244600.0   
2021-05-18  3787500.0   5426300.0  11098100.0  19394600.0   63342900.0   
2021-05-19  2870400.0   5542700.0  13278900.0  11624900.0   92612000.0   
2021-05-20  5083100.0   7067300.0  10423500.0  10178700.0   76756000.0   
2021-05-21  1974771.0   1823781.0   4861394.0   3663751.0   43314475.0   

Attributes price_change                                          
Symbols            BILI       PDD       JPM       WMT      AAPL  
Date                                                             
2019-05-22    -0.080001 -1.060001 -0.910004  1.110001 -0.955002  
2019-05-23    -0.370000  0.070002 -2.180000 -0.370003 -0.779999  
2019-05-24    -0.270000 -0.250000  1.070000  0.809998 -0.172501  
2019-05-28    -0.040000 -0.050001 -1.190002 -0.250000 -0.185001  
2019-05-29    -0.230000 -0.250000 -0.299995 -0.299995 -0.212498  
...                 ...       ...       ...       ...       ...  
2021-05-17     1.369995 -0.310005  0.660004 -0.630005 -1.180000  
2021-05-18     2.820000  4.220001 -2.319992  3.020004 -1.419998  
2021-05-19     1.400002  4.930000 -1.240005  0.009995 -0.159996  
2021-05-20     0.340004  3.

## RSI indicator

Use the Relative Strength Index (RSI) indicator to interpret whether the stock has been overbought or oversold. Gerneally, RSI is ranging from 0 to 100, and an RSI above 70 indicates the stock was overvalued and an RSI below 30 would imply that it is undervalued.

In [4]:
# calculate the 14 day RSI, which is a general standard
n = 14

# make a copy of up and down day twice
up_df, down_df = stocks['price_change'].copy(), stocks['price_change'].copy()

# if the price in up days is lower than 0 set it to 0
# if the price in down days is greater than 0 set it tp 0
for symbol in stocks.columns.get_level_values(1)[:5]:
    up_df.loc[(up_df.loc[:, symbol] < 0), symbol] = 0
    down_df.loc[(down_df.loc[:, symbol] > 0), symbol] = 0

# convert the price change to absolute value
down_df = down_df.abs()

In [5]:
# calculate the Exponential Weighted Moving Average 
# in order to focus more on recent prices
ewma_up = up_df.ewm(span=n).mean()
ewma_down = down_df.ewm(span=n).mean()

# calculate the relative strength and RSI
relative_strength = ewma_up / ewma_down
relative_strength_index = 100 - 100 / (1 + relative_strength)

In [6]:
# append the down and up days and RSI into the original data frame
new_attrs = ['up_days', 'down_days', 'RSI']
attr_dfs = [up_df, down_df, relative_strength_index]

for attr, attr_df in zip(new_attrs, attr_dfs):
    for symbol in stocks.columns.get_level_values(1)[:5]:
        stocks[attr, symbol] = attr_df[symbol]

In [7]:
stocks

Attributes   Adj Close                                                  \
Symbols           BILI         PDD         JPM         WMT        AAPL   
Date                                                                     
2019-05-22   14.690000   20.459999  103.975410   98.809914   44.853859   
2019-05-23   14.320000   20.530001  101.930054   98.452286   44.088215   
2019-05-24   14.050000   20.280001  102.933960   99.235184   43.918888   
2019-05-28   14.010000   20.230000  101.817474   98.993553   43.737293   
2019-05-29   13.780000   19.980000  101.536003   98.703583   43.528706   
...                ...         ...         ...         ...         ...   
2021-05-17   98.239998  118.019997  164.669998  138.889999  126.269997   
2021-05-18  101.059998  122.239998  162.350006  141.910004  124.849998   
2021-05-19  102.459999  127.169998  161.110001  141.919998  124.690002   
2021-05-20  102.800003  131.000000  160.830002  142.419998  127.309998   
2021-05-21  102.419998  129.679993  162.250000  141.828293  125.569099   

Attributes       Close                                                  ...  \
Symbols           BILI         PDD         JPM         WMT        AAPL  ...   
Date                                                                    ...   
2019-05-22   14.690000   20.459999  110.820000  102.230003   45.695000  ...   
2019-05-23   14.320000   20.530001  108.639999  101.860001   44.915001  ...   
2019-05-24   14.050000   20.280001  109.709999  102.669998   44.742500  ...   
2019-05-28   14.010000   20.230000  108.519997  102.419998   44.557499  ...   
2019-05-29   13.780000   19.980000  108.220001  102.120003   44.345001  ...   
...                ...         ...         ...         ...         ...  ...   
2021-05-17   98.239998  118.019997  164.669998  138.889999  126.269997  ...   
2021-05-18  101.059998  122.239998  162.350006  141.910004  124.849998  ...   
2021-05-19  102.459999  127.169998  161.110001  141.919998  124.690002  ...   
2021-05-20  102.800003  131.000000  160.830002  142.419998  127.309998  ...   
2021-05-21  102.419998  129.679993  162.250000  141.828293  125.569099  ...   

Attributes down_days                                                RSI  \
Symbols         BILI       PDD       JPM       WMT      AAPL       BILI   
Date                                                                      
2019-05-22  0.080001  1.060001  0.910004  0.000000  0.955002   0.000000   
2019-05-23  0.370000  0.000000  2.180000  0.370003  0.779999   0.000000   
2019-05-24  0.270000  0.250000  0.000000  0.000000  0.172501   0.000000   
2019-05-28  0.040000  0.050001  1.190002  0.250000  0.185001   0.000000   
2019-05-29  0.230000  0.250000  0.299995  0.299995  0.212498   0.000000   
...              ...       ...       ...       ...       ...        ...   
2021-05-17  0.000000  0.310005  0.000000  0.630005  1.180000  45.757666   
2021-05-18  0.000000  0.000000  2.319992  0.000000  1.419998  51.017997   
2021-05-19  0.000000  0.000000  1.240005  0.000000  0.159996  53.595855   
2021-05-20  0.000000  0.000000  0.279999  0.000000  0.000000  54.270265   
2021-05-21  0.380005  1.320007  0.000000  0.591705  1.740898  53.271834   

Attributes                                               
Symbols           PDD        JPM         WMT       AAPL  
Date                                                     
2019-05-22   0.000000   0.000000  100.000000   0.000000  
2019-05-23   7.080385   0.000000   72.222084   0.000000  
2019-05-24   5.481161  29.372621   83.675983   0.000000  
2019-05-28   5.209613  21.332043   72.961921   0.000000  
2019-05-29   4.051612  19.758720   61.974648   0.000000  
...               ...        ...         ...        ...  
2021-05-17  32.368954  74.352166   47.582070  42.641157  
2021-05-18  42.725389  61.987375   61.541970  38.037418  
2021-05-19  52.525048  56.221315   61.581040  37.510905  
2021-05-20  58.838161  54.890991   63.709253  50.465990  
2021-05-21  55.882982  60.377297   59.229146  43.54

## Stochastic Oscillator
It measures the level of closing price relative to the low-high range changing over the period of time

In [16]:
n = 14

low_14, high_14 = stocks['Low'].copy(), stocks['High'].copy()

# calculate the stochastic oscillator
low_14 = low_14.rolling(window=n).min()
high_14 = high_14.rolling(window=n).max()

k_percent = 100 * (stocks['Close'] - low_14) / (high_14 - low_14)

Symbols,BILI,PDD,JPM,WMT,AAPL
Date,,,,,
2019-05-22,NaN,NaN,NaN,NaN,NaN
2019-05-23,NaN,NaN,NaN,NaN,NaN
2019-05-24,NaN,NaN,NaN,NaN,NaN
2019-05-28,NaN,NaN,NaN,NaN,NaN
2019-05-29,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2021-05-17,36.411459,17.770125,98.661652,44.842406,27.125470
2021-05-18,45.971294,32.311694,76.422202,69.727937,17.543841
2021-05-19,59.821135,54.458702,67.140725,69.841253,20.642986


In [17]:
# add the dataframe to the stocks 
new_attrs = ['low_14', 'high_14', 'k_percent']
attr_dfs = [low_14, high_14, k_percent]

for attr, attr_df in zip(new_attrs, attr_dfs):
    for symbol in stocks.columns.get_level_values(1)[:5]:
        stocks[attr, symbol] = attr_df[symbol]

## William %R
Similar to the Stochastic Oscillator. Ranging from -100 to 0. When its value is above -20, it indicates a sell signal and when its value is below -80, it indicates a buy signal.

In [18]:
r_percent = -100 * (high_14 - stocks['Close']) / (high_14 - low_14)

# add new dataframe to the stocks
for symbol in stocks.columns.get_level_values(1)[:5]:
    stocks['r_percent', symbol] = r_percent[symbol]

Attributes  Adj Close                                                \
Symbols          BILI        PDD         JPM         WMT       AAPL   
Date                                                                  
2019-05-22  14.690000  20.459999  103.975410   98.809914  44.853859   
2019-05-23  14.320000  20.530001  101.930054   98.452286  44.088215   
2019-05-24  14.050000  20.280001  102.933960   99.235184  43.918888   
2019-05-28  14.010000  20.230000  101.817474   98.993553  43.737293   
2019-05-29  13.780000  19.980000  101.536003   98.703583  43.528706   
2019-05-30  13.890000  19.650000  100.447632   98.771240  43.754475   
2019-05-31  13.500000  19.420000   99.415573   98.046341  42.961842   
2019-06-03  13.480000  19.030001   99.884697   98.548935  42.527481   
2019-06-04  14.110000  19.280001  102.962105   99.128876  44.083302   
2019-06-05  13.990000  19.129999  103.328033  100.926628  44.794956   
2019-06-06  14.170000  19.370001  103.553223  101.593559  45.452625   
2019-06-07  14.800000  19.780001  102.417946  102.511765  46.662441   
2019-06-10  15.060000  20.200001  103.525055  103.922935  47.258759   
2019-06-11  14.920000  20.049999  103.844063  104.328888  47.805996   
2019-06-12  14.690000  20.410000  102.521126  105.179436  47.653847   
2019-06-13  14.480000  20.469999  102.774467  105.015129  47.644032   
2019-06-14  14.270000  20.510000  103.037178  105.421082  47.298027   
2019-06-17  14.360000  20.590000  102.474243  105.508080  47.580223   
2019-06-18  15.160000  20.889999  103.872200  105.981689  48.699249   
2019-06-19  15.190000  20.959999  103.121620  105.952682  48.556911   
2019-06-20  15.580000  20.950001  103.384331  106.629257  48.947098   
2019-06-21  15.440000  21.049999  102.680641  107.412155  48.780224   
2019-06-24  16.070000  20.750000  101.948830  107.518471  48.731144   
2019-06-25  15.640000  20.540001  101.104408  107.015869  47.992500   
2019-06-26  15.970000  20.450001  101.779945  106.474625  49.030533   
2019-06-27  16.100000  20.760000  102.117699  106.416626  49.015808   
2019-06-28  16.270000  20.629999  104.894897  106.793579  48.569180   
2019-07-01  16.480000  20.639999  106.658775  106.919212  49.459972   
2019-07-02  16.610001  20.309999  106.771355  107.866440  49.749546   
2019-07-03  16.629999  20.090000  106.601288  108.562347  50.161819   
2019-07-05  16.740000  20.290001  107.234352  108.233727  50.117641   
2019-07-08  16.559999  20.040001  106.648529  108.948975  49.084515   
2019-07-09  16.600000  20.480000  107.102066  109.103615  49.383907   
2019-07-10  16.590000  19.840000  106.790253  109.200272  49.872242   
2019-07-11  16.469999  19.340000  107.810730  110.108810  49.509064   
2019-07-12  16.400000  19.139999  108.944588  110.766068  49.889423   
2019-07-15  16.450001  19.350000  107.621742  111.133354  50.358139   
2019-07-16  16.070000  20.120001  108.774506  110.920723  50.183903   
2019-07-17  15.580000  19.969999  107.706787  110.766068  49.901699   
2019-07-18  15.530000  20.190001  108.349304  110.882057  50.468563   
2019-07-19  15.540000  20.049999  107.281593  110.089493  49.715195   
2019-07-22  15.560000  20.139999  107.971352  109.045616  50.851383   
2019-07-23  15.710000  20.990000  109.946167  108.340050  51.248924   
2019-07-24  15.800000  20.930000  110.390259  108.253059  51.207211   
2019-07-25  15.700000  21.510000  109.331978  108.465698  50.802307   
2019-07-26  15.960000  21.610001  109.813873  109.238930  50.978992   
2019-07-29  16.129999  21.910000  109.464264  108.514030  51.455059   
2019-07-30  15.490000  21.950001  109.218597  108.311050  51.234207   
2019-07-31  15.150000  22.270000  109.606003  106.687256  52.279598   
2019-08-01  14.750000  22.660000  106.714676  105.720688  51.148312   

Attributes      Close                                                ...  \
Symbols          BILI        PDD         JPM         WMT       AAPL  ...   
Date                                                                 ...   
2019-05-